In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


This is the main Sensor Kennel application that will 
identify, read and write any sensor that's plugged in.

On the lower side:  
  Pin1 (left) Gnd, Pin3 3v3, P4 (Pin-3 from right) SDA, P15 (Pin-5 from right) SCL.
  P17 (Pin-2) TX, P13 (Upper side Pin-2) RX 
  For BNO055 with PS1=High SCL=TX(17), SDA=RX(13)
  
Do we want to draw a graph of numbers if a button is pressed?

Should we have MQTT numbers going out from it?

Or just keep it comprehensive

Sensors known:
* VL53L0X lidar
* VL6180 lidar
* SI7021 humidity
* TSL561 luminous
* TMP102 temp
* SDOF GyAccMag
* MLX90614 ir-temp
* BME280 barhumid
* SHT31D tmphumid
* MLX90621 16x4-ir
* PX4PITOT airspeed
* BNO055 orient
* GPS serial


In [3]:
print(globals())

{'uos': <module 'uos'>, 'fbuff': <FrameBuffer>, 'ds18x20': <module 'ds18x20'>, 'pled': Pin(25), 'DS18B20_task': <generator>, 'uart': UART(1, baudrate=9600, bits=8, parity=None, stop=1, tx=17, rx=13, rts=-1, cts=-1, txbuf=256, rxbuf=256, timeout=0, timeout_char=2), 'dallasobj': <DS18X20 object at 3ffc6570>, 'n': 4, 'gc': <module 'gc'>, 'doublepixels': <function doublepixels at 0x3ffc2860>, 'fatntext': <function fatntext at 0x3ffc2850>, 'plotdevices': <generator>, 'TMP102_task': <generator>, 'FMQTTQueue': <class 'FMQTTQueue'>, 'time': <module 'utime'>, 'dallasscanned': [bytearray(b'(\xaa\x1c\xf17\x14\x01\x03')], 'oledshow': <function oledshow at 0x3ffc24a0>, 'IdentifyUARTDevice': <function IdentifyUARTDevice at 0x3ffc2fb0>, 'i2c': <I2C>, 'devices': ['DS18B20 n=1'], 'TSL561_task': <generator>, 'devdesc': 'n=1', 'onewire': <module 'onewire'>, 'machine': <module 'umachine'>, 'IdentifyI2CDevice': <function IdentifyI2CDevice at 0x3ffc2fe0>, 'SHT31D_task': <generator>, 'Pin': <class 'Pin'>, 'I

In [6]:
help(dallasobj)

object <DS18X20 object at 3ffc6570> is of type DS18X20
  __init__ -- <function __init__ at 0x3ffc7050>
  convert_temp -- <function convert_temp at 0x3ffc7140>
  scan -- <function scan at 0x3ffc7040>
  __qualname__ -- DS18X20
  read_scratch -- <function read_scratch at 0x3ffc7170>
  write_scratch -- <function write_scratch at 0x3ffc7180>
  read_temp -- <function read_temp at 0x3ffc7190>
  __module__ -- ds18x20


In [8]:
print(dallasobj.scan())

[bytearray(b'(\xaa\x1c\xf17\x14\x01\x03')]


In [11]:
help(fbuff)

object <FrameBuffer> is of type FrameBuffer
  fill -- <function>
  fill_rect -- <function>
  pixel -- <function>
  hline -- <function>
  vline -- <function>
  rect -- <function>
  line -- <function>
  blit -- <function>
  scroll -- <function>
  text -- <function>


In [13]:
fbuff.fill_rect(10,10,10,10,0)
oledshow()

In [10]:
print(dallasobj.read_temp(b'(\xaa\x1c\xf17\x14\x01\x03'))

14.125


In [ ]:
        dallasobj.convert_temp()
        await asyncio.sleep_ms(800)
        fbuff.fill(0)
        fbuff.text("DS18B20 temps", 0, 0, 1)
        for i, d in enumerate(dallasscanned):
            try:
                t = dallasobj.read_temp(d)


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [6]:
help("modules")

__main__          framebuf          socket            upip
_boot             gc                ssl               upip_utarfile
_onewire          hashlib           struct            upysh
_thread           heapq             sys               urandom
_webrepl          inisetup          time              ure
apa106            io                ubinascii         urequests
array             json              ucollections      uselect
binascii          machine           ucryptolib        usocket
btree             math              uctypes           ussl
builtins          micropython       uerrno            ustruct
cmath             neopixel          uhashlib          utime
collections       network           uhashlib          utimeq
dht               ntptime           uheapq            uzlib
ds18x20           onewire           uio               webrepl
errno             os                ujson             webrepl_setup
esp               random            umqtt/robust      websocket
esp32    

In [32]:
%sendtofile --binary --source OLED_driver.py
%sendtofile --quiet --source device_detection.py
%sendtofile --quiet --source SI7021_funcs.py
%sendtofile --quiet --source TSL561_funcs.py
%sendtofile --quiet --source VL53L0X_funcs.py
%sendtofile --quiet --source VL6180_funcs.py
%sendtofile --quiet --source SDOF_funcs.py
%sendtofile --quiet --source SHT31D_funcs.py
%sendtofile --quiet --source BME280_funcs.py
%sendtofile --quiet --source MLX90621_funcs.py
%sendtofile --quiet --source BNO055serial_funcs.py
%sendtofile --quiet --source MLX90614_funcs.py
%sendtofile --quiet --source MS5611_funcs.py
%sendtofile --quiet --source OLED_grapher.py


Sent 3820 bytes in 128 chunks to OLED_driver.py.
Sent 109 lines (3180 bytes) to device_detection.py.
Sent 55 lines (2040 bytes) to SI7021_funcs.py.
Sent 54 lines (1757 bytes) to TSL561_funcs.py.
Sent 33 lines (975 bytes) to VL53L0X_funcs.py.
Sent 106 lines (3190 bytes) to VL6180_funcs.py.
Sent 42 lines (1800 bytes) to SDOF_funcs.py.
Sent 43 lines (1090 bytes) to SHT31D_funcs.py.
Sent 76 lines (3165 bytes) to BME280_funcs.py.
Sent 170 lines (5915 bytes) to MLX90621_funcs.py.
Sent 87 lines (2701 bytes) to BNO055serial_funcs.py.
Sent 27 lines (801 bytes) to MLX90614_funcs.py.
Sent 46 lines (1375 bytes) to MS5611_funcs.py.
Sent 145 lines (4083 bytes) to OLED_grapher.py.


In [33]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/

# This requires you to checkout https://github.com/peterhinch/micropython-async
%sendtofile --quiet --source /home/julian/extrepositories/micropython-async/asyn.py
%sendtofile --quiet --source /home/julian/extrepositories/micropython-async/aswitch.py

# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --quiet --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

# This is the wrapper code on the asynchronous mqtt
%sendtofile --quiet --source fmqttqueue.py

Sent 258 lines (8400 bytes) to lib/uasyncio/__init__.py.
Sent 315 lines (9509 bytes) to lib/uasyncio/core.py.
Sent 94 lines (2685 bytes) to lib/uasyncio/queues.py.
Sent 28 lines (846 bytes) to lib/uasyncio/synchro.py.
Sent 473 lines (14783 bytes) to asyn.py.
Sent 219 lines (8811 bytes) to aswitch.py.
Sent 638 lines (23042 bytes) to mqtt_as.py.
Sent 72 lines (2314 bytes) to fmqttqueue.py.


In [68]:
%sendtofile config.txt

wifiname     DoESLiverpool-MQTT
wifipassword decafbad00
mqttbroker   10.3.141.1
pinled       25
boardname    esp32black

Sent 5 lines (119 bytes) to config.txt.


In [49]:
%sendtofile config.txt

wifiname     DoESLiverpool-MQTT
wifipassword decafbad00
mqttbroker   10.3.141.1
pinled       25
boardname    heltecwhite


Sent 5 lines (121 bytes) to config.txt.


In [56]:
%sendtofile config.txt

wifiname     DoESLiverpool-MQTT
wifipassword decafbad00
mqttbroker   10.3.141.1
pinled       25
boardname    heltec1


Sent 5 lines (117 bytes) to config.txt.


In [511]:
%sendtofile config.txt

wifiname     mqtt-wifi
wifipassword decafbad00
mqttbroker   10.3.141.1
pinled       25
boardname    ttgo2


Sent 5 lines (106 bytes) to config.txt.


In [175]:
%sendtofile config.txt

wifiname     mqtt-wifi
wifipassword decafbad00
mqttbroker   10.3.141.1
pinled       2
boardname    esp32co2


Sent 5 lines (108 bytes) to config.txt.


In [67]:
print(open("config.txt").read())

wifiname     mqtt-wifi
wifipassword decafbad00
mqttbroker   10.3.141.1
pinled       25
boardname    esp32black


In [66]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB7 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 9, in <module>
  File "fmqttqueue.py", line 1, in <module>
  File "mqtt_as.py", line 28, in <module>
KeyboardInterrupt: 
MicroPython v1.10-227-g696549d2e on 2019-03-17; ESP32 module with ESP32
Type "help()" for more information.
>>> ^L1^!^NZ^H@J1!H)^F!!^H
Traceback (most recent call last):
  File "<stdin>", line 1
SyntaxError: invalid syntax
>>> ^Y`B^H1GL
Traceback (most recent call last):
  File "<stdin>", line 1
SyntaxError: invalid syntax
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [65]:
%sendtofile main.py

# connect to the OLED and initialize (and get its i2c bus)
from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext
import machine, time

from device_detection import IdentifyI2CDevice, IdentifyUARTDevice

from machine import Pin, I2C
import uasyncio as asyncio
from fmqttqueue import FMQTTQueue

fconfig = dict(x.split()  for x in open("config.txt"))
pled = Pin(int(fconfig["pinled"]), Pin.OUT)

# change baud for some 
for n in range(5):
    devices = IdentifyI2CDevice(i2c)
    print("devices", devices)
    if devices:
        break
    time.sleep(0.3)

if not devices:
    uart = machine.UART(1, baudrate=9600, rx=13, tx=17)
    # gps is 9600, bluefly is 57600  
    devices = IdentifyUARTDevice(uart)
if not devices:
    import onewire, ds18x20
    pindallas = machine.Pin(13, Pin.OUT, Pin.PULL_UP)
    dallasobj = ds18x20.DS18X20(onewire.OneWire(pindallas))
    dallasscanned = dallasobj.scan()
    if dallasscanned:
        sdallasqorder = {347:0, 376:1, 448:2, 475:3, 576:4, 311:5, 406:6, 157:7}
        dallasscanned.sort(key=lambda X: sdallasqorder.get(sum(X), 0))
        devices = ["DS18B20 n=%d"%len(dallasscanned)]
    print("dallas", devices)
        
if not devices:
    devices.append("nothing found")

# this might look like a lot of code, but it's the complete 
# visual output of text and numbers that we do per sensor here
devnames = [ ]
for dev in devices:
    devname, devdesc = dev.split(" ", 1)
    devnames.append(devname)
    fbuff.fill(0)
    fbuff.text(devname, 0, 0, 1)
    fbuff.text(devdesc, 0, 12, 1)
    doublepixels()
    oledshow()
    time.sleep(1)

async def plotdevices(fqm):
    topic = fconfig["boardname"]+"/"+devname
    if devname == "VL53L0X":
        from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
        VL53L0Xinit(i2c)
        while True:
            d = VL53L0Xdist()
            fbuff.fill(0)
            fbuff.text("VL53L0X", 0, 0, 1)
            fbuff.text("dist(mm):", 8, 20, 1)
            fatntext("%d" % d, 40, 36)
            oledshow()
            fqm.fput(topic, "%d"%d)
            await asyncio.sleep_ms(50)
        
    if devname == "VL6180":
        from VL6180_funcs import VL6180init, distmm
        VL6180init(i2c)
        while True:
            d = distmm()
            fbuff.fill(0)
            fbuff.text("VL6180", 0, 0, 1)
            fbuff.text("dist(mm):", 8, 20, 1)
            fatntext("%d" % d, 40, 36)
            oledshow()
            fqm.fput(topic, "%d" % d)
            await asyncio.sleep_ms(50)

    if devname == "BME280" or devname == "BME180":
        from BME280_funcs import bme280init, readBME280
        bme280init(i2c)
        topictemp = topic+"/temp"
        topichumid = topic+"/humid"
        topicpressure = topic+"/pressure"
        n = 0
        while True:
            temp, pressure, humid = readBME280()
            fbuff.fill(0)
            fbuff.text("BME280", 0, 0, 1)
            fbuff.text("tmp:", 0, 16, 1)
            fatntext("%.2f"%(temp), 32, 10)
            fbuff.text("prs:", 0, 36, 1)
            fatntext("%.2f"%(pressure), 32, 30)
            fbuff.text("hum:", 0, 54, 1)
            fatntext("%.2f"%(humid), 32, 50)
            oledshow()
            if (n % 5) == 0:
                fqm.fput(topichumid, "%.2f"%humid)
                fqm.fput(topictemp, "%.2f"%temp)
                fqm.fput(topicpressure, "%.2f"%pressure)
            n += 1
            await asyncio.sleep_ms(110)

    if devname == "SDOF":
        from SDOF_funcs import SetupAccGyrMag, readvectorsensor
        SetupAccGyrMag(i2c)
        topicaccx = topic+"/acc/x"
        topicaccy = topic+"/acc/y"
        topicaccz = topic+"/acc/z"
        n = 0
        while True:
            fbuff.fill(0)
            ax, ay, az = readvectorsensor("a")
            fbuff.text("Acc:", 0, 0, 1)
            fbuff.text("%d"%ax, 8, 8, 1)
            fbuff.text("%d"%ay, 8, 16, 1)
            fbuff.text("%d"%az, 8, 24, 1)
            x, y, z = readvectorsensor("g")
            fbuff.text("gyr:", 64, 0, 1)
            fbuff.text("%d"%x, 72, 8, 1)
            fbuff.text("%d"%y, 72, 16, 1)
            fbuff.text("%d"%z, 72, 24, 1)
            x, y, z = readvectorsensor("c")
            fbuff.text("mag:", 40, 32, 1)
            fbuff.text("%d"%x, 48, 40, 1)
            fbuff.text("%d"%y, 48, 48, 1)
            fbuff.text("%d"%z, 48, 56, 1)
            oledshow()
            
            if (n % 5) == 0:
                fqm.fput(topicaccx, "%d"%ax)
                fqm.fput(topicaccy, "%d"%ay)
                fqm.fput(topicaccz, "%d"%az)
            n += 1
            await asyncio.sleep_ms(100)
            
    if devname == "SI7021":
        from SI7021_funcs import setupSI7021, SI7021humiditytemp
        setupSI7021(i2c)
        topichumid = topic+"/humid"
        topictemp = topic+"/temp"
        n = 0
        while True:
            h, t = SI7021humiditytemp()
            fbuff.fill(0)
            fbuff.text("SI7021", 0, 0, 1)
            fbuff.text("humid:", 0, 23, 1)
            fatntext("%.1f"%h, 50, 16)
            fbuff.text("temp:", 0, 53, 1)
            fatntext("%.1f"%t, 50, 46)
            if fqm.rqueue:
                fbuff.text(fqm.rqueue.popleft()[1].decode(), 0, 32)
            oledshow()
            if (n % 4) == 0:
                fqm.fput(topichumid, "%.1f"%h)
                fqm.fput(topictemp, "%.1f"%t)
            n += 1
            await asyncio.sleep_ms(250)

    if devname == "MLX90614":
        from MLX90614_funcs import mlx90614temps
        from sevensegmentdisplay import encodeledstring # wired to heltecwhite
        i2c.init(scl=Pin(15), sda=Pin(4), freq=100000) # can't handle higher frequencies
        topicambient = topic+"/temp/ambient"
        topicir = topic+"/temp/ir"
        n = 0
        while True:
            tempAmbient, tempIR = mlx90614temps(i2c)
            if tempAmbient is not None:
                encodeledstring("%.2f c"% tempIR)
                if (n % 5) == 0 and tempIR > -10:
                    fqm.fput(topicambient, "%.2f"%tempAmbient)
                    fqm.fput(topicir, "%.2f"%tempIR)
                n += 1
            if fqm.rqueue:
                topic, msg = fqm.rqueue.popleft()
                encodeledstring(msg)
                await asyncio.sleep_ms(800)
            await asyncio.sleep_ms(200)
            
    if devname == "LP55231":
        fqm.addsubscription("LP55231/#")
        while True:
            if fqm.rqueue:
                topic, msg = fqm.rqueue.popleft()
                print((topic, msg))
                fbuff.fill(0)
                fbuff.text("LP55231", 0, 0, 1)
                fbuff.text("topic:", 0, 8, 1)
                fbuff.text(topic.decode(), 0, 16, 1)
                fbuff.text("msg:", 0, 53, 1)
                fatntext(msg.decode(), 50, 46)
                oledshow()
                try:
                    pos, col = topic.decode().split("/")[1:]
                    intensity = int(msg)
                    if col == "red":
                        bpos = 0x1C + int(pos)
                    else:
                        bpos = (0x16 if col == "green" else 0x17) + int(pos)*2
                    i2c.writeto(0x32, bytes((bpos, intensity)))
                except ValueError as e:
                    print(e)
            else:
                await asyncio.sleep_ms(5)

async def CDM7160_task():
    topic = fconfig["boardname"]+"/CDM7160"
    co2ppmFiltered = 0
    while True:
        try:
            k = i2c.readfrom_mem(0x69, 0x03, 2)
            co2ppm = (k[0]+k[1]*256)
        except OSError:
            co2ppm = None
        if co2ppm is not None and co2ppm != 0:
            pled.value(1)
            await asyncio.sleep_ms(50)
            pled.value(0)

            f = 0.02 if co2ppmFiltered else 1.0
            co2ppmFiltered = co2ppm*f + co2ppmFiltered*(1-f)
            fbuff.fill(0)
            fbuff.text("figaroCO2", 0, 0, 1)
            fbuff.text("ppm: ", 8, 20, 1)
            fbuff.text("%d" % co2ppm, 48, 20)
            fbuff.text("(unfiltered)", 128-12*8, 28)
            fatntext("%d" % int(co2ppmFiltered), 32, 48)
            oledshow()
            fqm.fput(topic, "%d" % co2ppm)
                
            await asyncio.sleep_ms(3000)
                
                
async def DS18B20_task():
    topicds = [ fconfig["boardname"]+"/DS18B20/"+str(i)  for i in range(len(dallasscanned)) ]
    while True:
        dallasobj.convert_temp()
        await asyncio.sleep_ms(800)
        fbuff.fill(0)
        fbuff.text("DS18B20 temps", 0, 0, 1)
        for i, d in enumerate(dallasscanned):
            try:
                t = dallasobj.read_temp(d)
            except Exception:
                t = -100
            if len(dallasscanned) != 1:
                fbuff.text("%.2f"%t, (i%2)*64, (i//2+2)*8, 1)
            else:
                fatntext("%.2f"%t, 8, 20)
            if t != -100:
                fqm.fput(topicds[i], "%.2f"%t)
        if fqm.rqueue:
            topic, msg = fqm.rqueue.popleft()
            fbuff.text(msg.decode(), 0, 56, 1)
        oledshow()

async def TMP102_task():
    topic = fconfig["boardname"]+"/TMP102"
    n = 0
    while True:
        try:
            r = i2c.readfrom_mem(0x48, 0x00, 2)
            t = r[0] + r[1]/256
            fbuff.fill(0)
            fbuff.text("TMP102", 0, 0, 1)
            fbuff.text("temp:", 0, 23, 1)
            fatntext("%.03f"%t, 30, 40)
            if fqm.rqueue:
                fbuff.text(fqm.rqueue.popleft()[1].decode(), 0, 56, 1)
            oledshow()
            if (n % 5) == 0:
                fqm.fput(topic, "%.03f"%t)
            n += 1
        except OSError as e:
            print(e, "TMP102_task")
            await asyncio.sleep_ms(1000)
        await asyncio.sleep_ms(100)
            
async def SHT31D_task():
    topichumid = fconfig["boardname"]+"/SHT31D/humid"
    topictemp = fconfig["boardname"]+"/SHT31D/temp"
    n = 0
    from SHT31D_funcs import SHT31Dsetup, readSHT31D
    SHT31Dsetup(i2c)
    while True:
        try:
            tmp, humid = readSHT31D()
            fbuff.fill(0)
            fbuff.text("SHT31-D", 0, 0, 1)
            fbuff.text("temperature:", 10, 10, 1)
            fatntext("%.2f"%tmp, 20, 22)
            fbuff.text("humidity:", 10, 40, 1)
            fatntext("%.2f"%humid, 20, 50)
            oledshow()
            if (n % 5) == 0:
                fqm.fput(topichumid, "%.2f"%humid)
                fqm.fput(topictemp, "%.2f"%tmp)
            n += 1
        except OSError as e:
            print(e, "SHT31D_task")
            await asyncio.sleep_ms(1000)
        await asyncio.sleep_ms(100)
            
async def TSL561_task():
    from TSL561_funcs import setupTSL561, luminosity, setregtimings
    topicbr = fconfig["boardname"]+"/TSL561/broad"
    topicir = fconfig["boardname"]+"/TSL561/ir"
    setupTSL561(i2c)
    setregtimings(True, 1)
    n = 0
    while True:
        try:
            br, ir = luminosity()
            fbuff.fill(0)
            fbuff.text("TSL561", 0, 0, 1)
            fbuff.text("broad:", 0, 23, 1)
            fatntext("%d"%br, 50, 16)
            fbuff.text("ir:", 0, 53, 1)
            fatntext("%d"%ir, 50, 46)
            oledshow()
            if (n % 10) == 0:
                fqm.fput(topicir, "%d"%ir)
                fqm.fput(topicbr, "%d"%br)
            n += 1
        except OSError as e:
            print(e, "TSL561_task")
            await asyncio.sleep_ms(1000)
        await asyncio.sleep_ms(50)
            
async def MS5611_task():
    from MS5611_funcs import ms5611setup, readMS5611
    from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
    scrollinit()
    topicbaro = fconfig["boardname"]+"/MS5611/baro"
    ms5611setup(i2c)
    n = 0
    while True:
        try:
            temp, pr = readMS5611()
            #fbuff.fill(0)
            #fbuff.text("TSL561", 0, 0, 1)
            #fbuff.text("temp:", 0, 23, 1)
            #fatntext("%f"%temp, 50, 16)
            #fbuff.text("prs:", 0, 35, 1)
            #fatntext("%f"%pr, 0, 46)
            addscrollgraph(pr, time.ticks_ms())
            plotscrollgraph(fbuff)
            oledshow()
            oledshow()
            if (n % 10) == 0:
                fqm.fput(topicbaro, "%f"%pr)
            n += 1
        except OSError as e:
            print(e, "MS5611_task")
            await asyncio.sleep_ms(1000)
        await asyncio.sleep_ms(10)
            
# setup and operation of the async loop
fqm = FMQTTQueue(pled)
fqm.setupmqttas(fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"])
fqm.addsubscription(fconfig["boardname"]+"/msg")

aloop = asyncio.get_event_loop()
aloop.create_task(fqm.fstreamtask())
if "TMP102" in devnames:
    aloop.create_task(TMP102_task())
if "SHT31D" in devnames:
    aloop.create_task(SHT31D_task())
if "TSL561" in devnames:
    aloop.create_task(TSL561_task())
if "MS5611" in devnames:
    aloop.create_task(MS5611_task())
if "CDM7160" in devnames:
    aloop.create_task(CDM7160_task())
if "DS18B20" in devnames:
    aloop.create_task(DS18B20_task())
    
aloop.run_until_complete(plotdevices(fqm))
aloop.run_forever()


Sent 382 lines (13101 bytes) to main.py.


In [64]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [16]:
from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
scrollinit()

In [19]:
import math, time
from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
from MS5611_funcs import ms5611setup, readMS5611
scrollinit()
for i in range(0, 10000, 10):
    temp, pr = readMS5611()
    addscrollgraph(pr, time.ticks_ms())
    plotscrollgraph(fbuff)
    oledshow()


.............

In [440]:
import machine
pbeep = machine.PWM(machine.Pin(2))


PWM(2, freq=5000, duty=512)


In [472]:
print(i2c.scan())

[60, 119]


In [475]:
ads = i2c.scan()
res = [ ]
if 0x77 in ads:
    k = i2c.readfrom_mem(0x77, 0xD0, 1)[0]
    if k == 0x60:
        res.append("BME280 barhumid")
    elif k == 0x55:
        res.append("BME180 barhumid")
    else:
        res.append("MS5611 baro")


Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
OSError: [Errno 19] ENODEV


In [ ]:




        
if devname == "MLX90621":
    fbuff.text("11", 50, 50, 1)
    oledshow()
    from MLX90621_funcs import measure, cellfillplot
    fbuff.text("22", 59, 50, 1)
    oledshow()
    while True:
        temperatures = measure()
        tmin = min(temperatures)
        tmax = max(tmin+0.001, max(temperatures))
        fbuff.fill(0)
        fbuff.text("MLX90621", 0, 0, 1)
        for i in range(64):
            cellfillplot(fbuff, i//4, 1+(3-(i%4)), ((temperatures[i] - tmin)/(tmax - tmin))**2)
        fbuff.text("min=%.3f"%tmin, 12, 48)
        fbuff.text("max=%.3f"%tmax, 12, 56)
        oledshow()

if devname == "PX4PITOT":
    while True:
        k = i2c.readfrom(0x28, 4)
        status = k[0] & 0xC0; 
        fbuff.fill(0)
        fbuff.text("PX4PITOT", 0, 0, 1)
        if status != 0xC0:
            rawpress = ((k[0] & 0x3F) << 8) | (k[1]) 
            rawtemp = (k[2] << 3) | (k[3] >> 5)
            psi = (rawpress)*(1.0/(0x3FFF*0.4)) - 1.25
            diffpressure = psi*6894.75728
            temp = (rawtemp)*(200.0/0x7FF) - 50
            #airspeed = math.sqrt(abs(2*getdiffpressure())/1.2);
            fbuff.text("diffpress:", 10, 10, 1)
            fatntext("%.2f"%diffpressure, 20, 22)
            fbuff.text("temp:", 10, 40, 1)
            fatntext("%.2f"%temp, 20, 50)
        else:
            fbuff.text("BAD", 20, 20, 1)
        oledshow()
        time.sleep(0.2)
        
if devname == "BNO055":
    from BNO055serial_funcs import InitBNO055, BNO055absorientation, BNO055calibstat
    InitBNO055(uart)
    while True:
        fbuff.fill(0)
        calibstat = None
        try:
            pitch, roll, orient = BNO055absorientation()
            calibstat = BNO055calibstat()
        except Exception as e:
            fbuff.text("BNO055", 0, 0, 1)
            fbuff.text(e.args[0], 0, 16, 1)
        if calibstat is not None:  
            fbuff.text("BNO055 :{0:08b}".format(calibstat), 0, 0, 1)
            fbuff.text("pitch:", 8, 16, 1)
            fatntext("%d"%int(pitch), 64, 10)
            fbuff.text("roll:", 16, 32, 1)
            fatntext("%d"%int(roll), 64, 28)
            fbuff.text("orient:", 0, 52, 1)
            fatntext("%d"%int(orient), 64, 48)
        oledshow()
        time.sleep(0.1)
        
if devname == "GPS":
    while True:
        x = uart.readline()
        if x is not None:
            fbuff.fill(0)
            fbuff.text("GPS serial", 0, 0, 1)
            x = x.strip()
            for i in range(len(x)//16 + 1):
                fbuff.text(x[i*16:i*16+16], 0, 15+i*8, 1)
            oledshow()
            time.sleep_ms(100)
        time.sleep_ms(50)


In [ ]:
        
# read any GPS serial signals on 
fbuff.text("Znone", 120, 56, 1)
oledshow()
time.sleep(2.0)

from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
scrollinit()
a = machine.ADC(machine.Pin(36))  # Pin4 from top left
a.atten(a.ATTN_11DB)
a.width(a.WIDTH_12BIT)
while True:
    v = a.read()
    addscrollgraph(v, time.ticks_ms())
    plotscrollgraph(fbuff)
    fbuff.text("ADCPin36",0,0,1)
    fbuff.text(str(v),0,8,1)
    oledshow()
time.sleep(0.1)
    

In [ ]:
%sendtofile main.py

from machine import Pin, PWM, ADC
import time

p2 = Pin(2, Pin.OUT)
pA = Pin(5, Pin.OUT)
pB = Pin(16, Pin.OUT)

a = ADC(Pin(34, Pin.IN))
a.width(ADC.WIDTH_12BIT)
a.atten(ADC.ATTN_11DB)

def gopos(pos):
    for i in range(100):
        d = (a.read() - pos)
        if abs(d) > 40:
            di = 1 if d < 0 else 0
            pA.value(di)
            pB.value(1-di)
            p2.value(1)
        else:
            p2.value(0)
            break
        time.sleep(0.01)
    print(a.read())

import random
while True:
    pos = random.uniform(500, 3500)
    gopos(pos)
    time.sleep(0.5)
